# Housing price prediction using Machine Learning

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
data=pd.read_csv("D:\Workshops\Machine Learning for Data Science With Python\Datasets\Housing Price.CSV")

In [ ]:
data.head()

# Data Cleaning

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

# Exploratory Analysis

In [ ]:
data.describe()

In [ ]:
sns.pairplot(data)
plt.plot()

In [ ]:
sns.heatmap(data.corr(),vmin=-1,vmax=1,annot=True)
plt.show()

# Machine Learning

In [ ]:
data.head()

In [ ]:
x=data.iloc[:,:5].values
y=data.iloc[:,5].values

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

# Linear Regression

In [ ]:
lm=LinearRegression()

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
lm.coef_

In [ ]:
data.columns[0:5]

In [ ]:
pd.DataFrame(lm.coef_,index=data.columns[0:5],columns=["Coefficient"])

In [ ]:
lm.intercept_

In [ ]:
lm.score(x_train,y_train)

In [ ]:
y_pred=lm.predict(x_test)

In [ ]:
x_with_constant=sm.add_constant(x_train)

In [ ]:
lm_sm=sm.OLS(y_train,x_with_constant)

In [ ]:
result=lm_sm.fit()

In [ ]:
print(result.summary())

In [ ]:
print(variance_inflation_factor(x_train,0))
print(variance_inflation_factor(x_train,1))
print(variance_inflation_factor(x_train,2))
print(variance_inflation_factor(x_train,3))
print(variance_inflation_factor(x_train,4))

In [ ]:
resid=y_train-lm.predict(x_train)

In [ ]:
sns.distplot(resid)
plt.show()

In [ ]:
sns.scatterplot(lm.predict(x_train),resid)
plt.show()

# KNN Regression

In [ ]:
Errors=[]
K=np.arange(1,21)

for k in K:
    model=KNeighborsRegressor(n_neighbors=k)
    cvals=np.sqrt(-cross_val_score(model,x_train,y_train,cv=10,scoring="neg_mean_squared_error"))
    Errors.append(cvals.mean())

In [ ]:
plt.plot(K,Errors,"rx-")
plt.show()

In [ ]:
knn = KNeighborsRegressor(n_neighbors=4)

In [ ]:
knn.fit(x_train, y_train)

In [ ]:
y_pred=lm.predict(x_test)

# Random Forest Regression

In [ ]:
params={"n_estimators":[100,200,300,400,500]}
model=RandomForestRegressor()
cval=KFold(n_splits=5)

In [ ]:
gsearch=GridSearchCV(model,params,cv=cval)

In [ ]:
results=gsearch.fit(x_train,y_train)
results.best_params_

In [ ]:
rf = RandomForestRegressor(n_estimators=500)

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
y_pred=rf.predict(x_test)

In [ ]:
sns.scatterplot(y_test,y_pred)
plt.plot()

In [ ]:
idx=np.argsort(rf.feature_importances_)

In [ ]:
sns.barplot(x=rf.feature_importances_[idx],y=data.columns[:5][idx])
plt.show()

# Stacking Regression

In [ ]:
bmodel1=LinearRegression()
bmodel2=KNeighborsRegressor(n_neighbors=6)

In [ ]:
metamodel=RandomForestRegressor(n_estimators=500)

In [ ]:
st=StackingRegressor(regressors=[bmodel1,bmodel2],meta_regressor=metamodel)

In [ ]:
st.fit(x_train, y_train)

In [ ]:
y_pred=st.predict(x_test)

In [ ]:
sns.scatterplot(y_test,y_pred)
plt.plot()

# Comparing performance

In [ ]:
y_pred=lm.predict(x_test)
lm_rmse=np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
y_pred=knn.predict(x_test)
knn_rmse=np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
y_pred=rf.predict(x_test)
rf_rmse=np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
y_pred=st.predict(x_test)
st_rmse=np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
pd.DataFrame({"Model":["Linear Regression","KNN","Random Forest","Stacking"],"RMSE":[lm_rmse,knn_rmse,rf_rmse,st_rmse]})